## Replica model

### Spatial reranking

Testing how to implement the spatial reranking.

In [1]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import sys

sys.path.insert(0, "../model/")
from utils import *
from metrics import *
from model_replica import ReplicaNet
from dataloader_replica import ReplicaDataset


path = '/home/guhennec/scratch/2021_Cini/TopologicalAnalysis_Cini/data/'
data_dir = '/scratch/students/schaerf/'
replica_dir='/mnt/project_replica/datasets/cini/'



2022-04-21 13:09:33.590690: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
data = pd.read_csv(data_dir + 'dedup_data_sample.csv').drop(columns=['Unnamed: 0', 'level_0'])
print(data.shape)
data.head()

(12862, 47)


,uid,cluster,index,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,...,CardboardURL,ImageURL,uid_connection,img1,img2,type,annotated,uid2,set,path
0,f2c17f970b114294a0403d5d4e59386f,f2c17f970b114294a0403d5d4e59386f,111077,74A,745,MITSUOKI Tosa,"Episodio del ""Genji Monogatari"" Dis.","MITSUOKI, Tosa","Mitsuoki, Tosa",ulan:500121170,...,https://dhlabsrv4.epfl.ch/iiif_cini/74A%2F74A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,Not uid connection,NaN,NaN,NaN,NaN,74A/74A_745.jpg,NaN,74A/74A_745.jpg
1,e165258d8e6142acac6d5617cc091b8a,e165258d8e6142acac6d5617cc091b8a,103106,114B,739,SORIA Giovanni Battista,Atrio.,"SORIA, Giovanni Battista","Soria, Giovanni Battista",ulan:500020294,...,https://dhlabsrv4.epfl.ch/iiif_cini/114B%2F114...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,Not uid connection,NaN,NaN,NaN,NaN,114B/114B_739.jpg,NaN,114B/114B_739.jpg
2,a266c68d59454209a9c0c3ec0e9bf99d,a266c68d59454209a9c0c3ec0e9bf99d,74427,39A,269,EVERDINGEN Allaert van,Paesaggio ecoa earri penna e pennello se 193 mm,EVERDINGEN Allaert van,"Everdingen, Allaert van",ulan:500115159,...,https://dhlabsrv4.epfl.ch/iiif_cini/39A%2F39A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,Not uid connection,NaN,NaN,NaN,NaN,39A/39A_269.jpg,NaN,39A/39A_269.jpg
3,4a617aa8a9cd4d89874221e2beec9931,4a617aa8a9cd4d89874221e2beec9931,34405,89A,468,SEC XVI,Leonendi San Marco.,SEC. XVI,NaN,NaN,...,https://dhlabsrv4.epfl.ch/iiif_cini/89A%2F89A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,Not uid connection,NaN,NaN,NaN,NaN,89A/89A_468.jpg,NaN,89A/89A_468.jpg
4,bba4f0d0294f420b90310cfe333c03ad,bba4f0d0294f420b90310cfe333c03ad,85957,84B,729,SEC XVI ex,Cherubino.,SEC. XVI ex,NaN,NaN,...,https://dhlabsrv4.epfl.ch/iiif_cini/84B%2F84B_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,Not uid connection,NaN,NaN,NaN,NaN,84B/84B_729.jpg,NaN,84B/84B_729.jpg


In [3]:
device = 'cpu'
model = ReplicaNet('resnext-101', device)
#model.load_state_dict(torch.load(data_dir + "model_weights_" + 'resnext-101'))
print(model)

ReplicaNet(
  (non_pooled): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
   

In [4]:
import torch.nn.functional as F

triplet_loss = nn.TripletMarginLoss(
    margin=0.0001, reduction='mean' # to be optimized margin
)

triplet_loss = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - F.cosine_similarity(x, y)
    )
    
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-6) # to be optimized lr and method
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.01) # to be optimized step and gamma


In [5]:
dts = {x: ReplicaDataset(data_dir + 'dataset/abc_' + x + '_100.csv', data_dir + 'deprecated/subset.csv', replica_dir, data_dir, x, 320) for x in ['train', 'val']}
dataset_sizes = {x: len(dts[x]) for x in ["train", "val"]}
train_dataloaders = {x: DataLoader(dts[x], batch_size=1, shuffle=False) for x in ["train", "val"]}

In [6]:
for i, list_ in tqdm(enumerate(train_dataloaders['train'])):
    
    [a, b, c] = list_
    a = a.squeeze(1).to(device)
    b = b.squeeze(1).to(device)
    c = c.squeeze(1).to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

                    
    # Forward pass
    a_np, b_np, c_np, A, B, C = model.non_pooled_forward(a, b, c)

    print(a_np.shape, A.shape)

    break

0it [00:02, ?it/s]

torch.Size([1, 2048, 10, 10]) torch.Size([1, 2048, 1, 1])


In [7]:
from time import time


class Timer:

    def __init__(self, description, disable=False):
        self.description = description
        self.disable = disable

    def __enter__(self):
        self.start = time()
        return self

    def __exit__(self, *args):
        self.end = time()
        self.interval = self.end - self.start
        if not self.disable:
            print("{} : {}".format(self.description, self.interval))


In [8]:
a = np.moveaxis(a_np.squeeze(0).detach().numpy(), 0, -1)
c = np.moveaxis(c_np.squeeze(0).detach().numpy(), 0, -1)
b = np.moveaxis(b_np.squeeze(0).detach().numpy(), 0, -1) # takes np array instead of tensors and different shape
a.shape

(10, 10, 2048)

In [9]:
def normalize(f_map, norm_epsilon):
    for i in range(f_map.shape[0]):
        for j in range(f_map.shape[1]):
            f_map[i, j, :] /= np.linalg.norm(f_map[i, j]) + norm_epsilon


def nb_unravel(ind, dims):
    result = np.empty(len(dims), dtype=np.int32)
    offsets = np.empty(len(dims), dtype=np.int32)
    d = len(dims)
    o = 1
    for i in range(d):
        offsets[d-1-i] = o
        o *= dims[d-1-i]
    for i, o in enumerate(offsets):
        result[i] = ind // o
        ind = ind % o
    return result

def nb_unravel_array(inds, dims): # not sure what this tries to do
    result = np.empty((len(inds), len(dims)), dtype=np.int32)
    offsets = np.empty(len(dims), dtype=np.int32)
    d = len(dims)
    o = 1
    for i in range(d):
        offsets[d-1-i] = o
        o *= dims[d-1-i]
    for k in range(len(inds)):
        ind = inds[k]
        for i, o in enumerate(offsets):
            result[k, i] = ind // o
            ind = ind % o
    return result


def get_candidates(f_map_1, f_map_2, norm_epsilon=0, margin=1, crosscheck_limit=2):
    normalize(f_map_1, norm_epsilon) # can easily be added into a model
    normalize(f_map_2, norm_epsilon)

    h1, w1, d_size = f_map_1.shape

    h2, w2, _ = f_map_2.shape
    # Convert to descriptors, keypoint versions
    des1 = np.ascontiguousarray(f_map_1[margin:h1 - margin, margin:w1 - margin]).reshape((-1, d_size)) #remove a margin: from 10x10 -> 8x8
    des2 = np.ascontiguousarray(f_map_2[margin:h2 - margin, margin:w2 - margin]).reshape((-1, d_size))
    # print(des1.shape) 64x2048
    kp1 = (nb_unravel_array(np.arange(len(des1)), (h1 - 2 * margin, w1 - 2 * margin)) + 0.5 + margin).astype(np.float32) # why +1.5?
    kp2 = (nb_unravel_array(np.arange(len(des2)), (h2 - 2 * margin, w2 - 2 * margin)) + 0.5 + margin).astype(np.float32)
    # print(kp1.shape) 64x2
    # print(kp1[0]) 1.5 1.5
    # Because of the margin, the arrays might be empty
    if len(kp1) == 0 or len(kp2) == 0:
        return np.empty((0, 2), np.float32), np.empty((0, 2), np.float32),  np.empty((0,), np.float32)
    # d = distance.cdist(des1, des2)
    d = 1 - des1 @ des2.T  # Warning, that is ~1/2 of the euclidean distance since des1.norm ~ des2.norm ~ 1
    # computes inner product distance between all 64 2048 dimensional descriptors and all 64 others
    # print(d.shape) 64x64
    #print(d[0])
    best_1 = np.empty((len(d), crosscheck_limit), dtype=np.int32)
    for i in range(len(d)):
        best_1[i, :] = np.argsort(d[i, :])[:crosscheck_limit] # for each row get two most similar ones
    d_T = d.T
    best_2 = np.empty((len(d_T), crosscheck_limit), dtype=np.int32)
    for i in range(len(d_T)):
        best_2[i, :] = np.argsort(d_T[i, :])[:crosscheck_limit] # for each column get two most similar 

    # best_1 = np.argsort(d, axis=1)[:, :crosscheck_limit]
    # best_2 = np.argsort(d.T, axis=1)[:, :crosscheck_limit]
    best_1_o = [set(best_1[i, :]) for i in range(len(best_1))]
    best_2_o = [set(best_2[i, :]) for i in range(len(best_2))]
    # d = des1 @ des2.T
    # best_1 = np.argmax(d, axis=1)
    # best_2 = np.argmax(d, axis=0)
    good = np.array([(i, j) for i, s in enumerate(best_1_o) for j in s if i in best_2_o[j]])
    
    src_pts = kp1[good[:, 0]]
    dst_pts = kp2[good[:, 1]]
    distances = np.array([d[good[i, 0], good[i, 1]] for i in range(len(good))])
    return src_pts, dst_pts, distances


In [10]:
src_pts, dst_pts, distances = get_candidates(a, b, norm_epsilon=0.00001)
src_pts, dst_pts, distances, len(src_pts)

(array([[1.5, 1.5],
        [1.5, 2.5],
        [1.5, 2.5],
        [2.5, 1.5],
        [2.5, 2.5],
        [2.5, 2.5],
        [2.5, 3.5],
        [2.5, 4.5],
        [2.5, 4.5],
        [2.5, 5.5],
        [2.5, 5.5],
        [2.5, 7.5],
        [3.5, 1.5],
        [3.5, 6.5],
        [3.5, 6.5],
        [3.5, 7.5],
        [4.5, 4.5],
        [4.5, 7.5],
        [5.5, 2.5],
        [5.5, 2.5],
        [5.5, 3.5],
        [5.5, 3.5],
        [5.5, 4.5],
        [6.5, 2.5],
        [6.5, 5.5],
        [6.5, 5.5],
        [6.5, 6.5],
        [7.5, 2.5],
        [7.5, 2.5],
        [7.5, 4.5],
        [7.5, 5.5],
        [7.5, 7.5],
        [7.5, 8.5],
        [8.5, 5.5],
        [8.5, 8.5]], dtype=float32),
 array([[1.5, 1.5],
        [1.5, 7.5],
        [2.5, 7.5],
        [2.5, 1.5],
        [2.5, 1.5],
        [2.5, 2.5],
        [2.5, 6.5],
        [2.5, 5.5],
        [3.5, 5.5],
        [3.5, 4.5],
        [2.5, 5.5],
        [3.5, 2.5],
        [4.5, 1.5],
        [4.5, 4.5],
   

In [11]:

def spatially_coherent_mask(src_pts, dst_pts, residual_threshold=2.0):
    min_x0_y0, increment_x0_y0 = -15, 1 # gets possible modifications
    max_x0_y0 = -min_x0_y0
    possible_x0 = np.arange(min_x0_y0, max_x0_y0, increment_x0_y0, np.int32) # -15, -14, ... , 15
    possible_y0 = np.arange(min_x0_y0, max_x0_y0, increment_x0_y0, np.int32)
    possible_lambdas = np.exp(np.arange(-7, 7)*0.2)  # ln(4) ~ 1.4 so 0.25-4x zoom
    possible_params = np.zeros((len(possible_x0), len(possible_y0), len(possible_lambdas), 2)) # Dx, Dy, lambda, flip
    for i in range(len(src_pts)):
        src_y, src_x = src_pts[i]
        dst_y, dst_x = dst_pts[i]
        for i_lamb, lamb in enumerate(possible_lambdas): # lambda is shift?
            x0 = dst_x-lamb*src_x
            y0 = dst_y-lamb*src_y
            i_x0 = int(round((x0-min_x0_y0)/increment_x0_y0))
            i_y0 = int(round((y0-min_x0_y0)/increment_x0_y0))
            if 0 <= i_x0 < len(possible_x0) and 0 <= i_y0 < len(possible_y0):
                possible_params[i_x0, i_y0, i_lamb, 0] += 1

            # Flip
            x0 = dst_x+lamb*src_x
            i_x0 = int(round((x0-min_x0_y0)/increment_x0_y0))
            if 0 <= i_x0 < len(possible_x0) and 0 <= i_y0 < len(possible_y0):
                possible_params[i_x0, i_y0, i_lamb, 1] += 1

    best_inds = np.argsort(possible_params.ravel())[-5:]
    best_inliers = 0
    best_mask = np.full(len(src_pts), False, dtype=np.bool_)
    best_M = np.array([
                [1, 0],
                [0, 1],
                [0, 0]
            ], dtype=np.float32)
    mask = best_mask.copy()
    preds = np.empty((1, 2), dtype=np.float32)
    src_pts_intercept = np.concatenate((src_pts, np.ones((len(src_pts), 1), dtype=np.float32)), axis=1)
    for ind in best_inds:
        i_x0, i_y0, i_lamb, is_flipped = nb_unravel(ind, dims=possible_params.shape)
        x0 = possible_x0[i_x0]
        y0 = possible_y0[i_y0]
        lamb = possible_lambdas[i_lamb]

        M = np.array([
                [lamb, 0],
                [0, (1-2*int(is_flipped))*lamb],
                [y0, x0]
            ], dtype=np.float32)
        preds = src_pts_intercept @ M
        mask = np.sum(np.square(preds-dst_pts), axis=1) <= residual_threshold
        if np.sum(mask) > best_inliers:
            best_inliers = np.sum(mask)
            best_mask[:] = mask
            best_M = M

    assert np.sum(best_mask) == best_inliers, "weird"

    if best_inliers > 0:
        assert np.sum(best_mask) > 0, "weird2"
        # Refine matrix
        #M, _, _, _ = np.linalg.lstsq(np.concatenate([src_pts[best_mask], np.ones((np.sum(best_mask), 1))], axis=1),
        #                             dst_pts[best_mask], rcond=-1)
        return best_M, best_mask
    else:
        print("No inliers?")
        return best_M, best_mask

In [12]:
M, mask = spatially_coherent_mask(src_pts, dst_pts, residual_threshold=2.0)
M, mask

(array([[ 1.,  0.],
        [ 0., -1.],
        [ 1., 10.]], dtype=float32),
 array([False,  True,  True, False, False, False,  True,  True,  True,
         True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False]))

In [13]:
def match_feature_maps(f_map_1, f_map_2, norm_epsilon=0, margin=1, crosscheck_limit=3):
    with Timer("candidates", disable=True):
        src_pts, dst_pts, distances = get_candidates(f_map_1, f_map_2, norm_epsilon, margin, crosscheck_limit)
    
    if len(src_pts) == 0:
        print("No candidates")

    with Timer("spatially_coherent", disable=True):
        M, mask = spatially_coherent_mask(src_pts, dst_pts, residual_threshold=2.0)
    num_matches = int(np.sum(mask))

    h1, w1, _ = f_map_1.shape
    h2, w2, _ = f_map_2.shape

    if num_matches == 0:
        return num_matches, None, (src_pts, dst_pts), mask.tolist(), ((0,0,1,1), (0,0,1,1))

    m1 = np.min(src_pts[mask], axis=0)
    m2 = np.max(src_pts[mask], axis=0)
    box1 = ((m1[0]-0.5)/h1, (m1[1]-0.5)/w1, (m2[0]-m1[0]+1)/h1, (m2[1]-m1[1]+1)/w1)
    m1 = np.min(dst_pts[mask], axis=0)
    m2 = np.max(dst_pts[mask], axis=0)
    box2 = ((m1[0]-0.5)/h2, (m1[1]-0.5)/w2, (m2[0]-m1[0]+1)/h2, (m2[1]-m1[1]+1)/w2)

    return num_matches, None, (src_pts, dst_pts), mask.tolist(), (box1, box2)

In [14]:
match_feature_maps(a, b, norm_epsilon=0.00001)

(62,
 None,
 (array([[1.5, 1.5],
         [1.5, 2.5],
         [1.5, 2.5],
         [1.5, 2.5],
         [1.5, 4.5],
         [2.5, 1.5],
         [2.5, 1.5],
         [2.5, 2.5],
         [2.5, 2.5],
         [2.5, 2.5],
         [2.5, 3.5],
         [2.5, 3.5],
         [2.5, 4.5],
         [2.5, 4.5],
         [2.5, 5.5],
         [2.5, 5.5],
         [2.5, 5.5],
         [2.5, 6.5],
         [2.5, 6.5],
         [2.5, 7.5],
         [3.5, 1.5],
         [3.5, 1.5],
         [3.5, 4.5],
         [3.5, 5.5],
         [3.5, 6.5],
         [3.5, 6.5],
         [3.5, 6.5],
         [3.5, 7.5],
         [3.5, 7.5],
         [4.5, 3.5],
         [4.5, 4.5],
         [4.5, 4.5],
         [4.5, 5.5],
         [4.5, 6.5],
         [4.5, 7.5],
         [4.5, 7.5],
         [4.5, 7.5],
         [4.5, 8.5],
         [5.5, 2.5],
         [5.5, 2.5],
         [5.5, 2.5],
         [5.5, 3.5],
         [5.5, 3.5],
         [5.5, 3.5],
         [5.5, 4.5],
         [5.5, 4.5],
         [5.5, 4.5],
 

In [15]:
match_feature_maps(a, c, norm_epsilon=0.00001)

(38,
 None,
 (array([[1.5, 1.5],
         [1.5, 4.5],
         [1.5, 4.5],
         [1.5, 5.5],
         [1.5, 5.5],
         [2.5, 4.5],
         [2.5, 4.5],
         [2.5, 4.5],
         [2.5, 5.5],
         [2.5, 5.5],
         [2.5, 5.5],
         [2.5, 6.5],
         [2.5, 6.5],
         [3.5, 3.5],
         [3.5, 3.5],
         [3.5, 4.5],
         [3.5, 4.5],
         [3.5, 5.5],
         [3.5, 5.5],
         [3.5, 5.5],
         [3.5, 6.5],
         [4.5, 2.5],
         [4.5, 2.5],
         [4.5, 3.5],
         [4.5, 3.5],
         [4.5, 3.5],
         [4.5, 4.5],
         [4.5, 4.5],
         [4.5, 5.5],
         [5.5, 1.5],
         [5.5, 2.5],
         [5.5, 3.5],
         [5.5, 3.5],
         [5.5, 3.5],
         [5.5, 4.5],
         [5.5, 5.5],
         [5.5, 5.5],
         [6.5, 2.5],
         [6.5, 2.5],
         [6.5, 3.5],
         [6.5, 3.5],
         [6.5, 4.5],
         [6.5, 4.5],
         [6.5, 5.5],
         [6.5, 6.5],
         [7.5, 2.5],
         [7.5, 2.5],
 

In [ ]:
def spatial_difference(x, y):

    # make each into 2048 equal 10x10 matrices  
    # count number of equal rounded 10x10 matrices between x and y 
    # ratio of 2048
    # 1 - ratio

    return ''

loss = loss_cosine_distance + loss_spatial_distance